In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/demo_distance.csv")

In [3]:
min_dist_df = df.pivot_table(
    index=["poll_id", "poll_lat", "poll_lon"],
    columns="aid_id",
    values="haversine",
    aggfunc=np.min,
).rename(columns=lambda col: f"{col}_min_dist")

min_dist_df.reset_index().rename_axis(None, axis=1).set_index("poll_id")

,poll_lat,poll_lon,p116170_min_dist,p125829_min_dist
poll_id,,,,
362725,-23.547403,-46.300659,34.172744,41.607432
583653,-23.547369,-46.358326,28.298120,37.446734


In [4]:
count_df = (
    df.loc[lambda df_: df_["haversine"] <= 60]
    .pivot_table(
        index=["poll_id", "poll_lat", "poll_lon"],
        columns="aid_id",
        values="haversine",
        aggfunc=len,
    )
    .rename(columns=lambda col: f"{col}_count")
    # polling locations are generally not close enough to all aid locations
    .fillna(0)
)

count_df.reset_index().rename_axis(None, axis=1).set_index("poll_id")

,poll_lat,poll_lon,p116170_count,p125829_count
poll_id,,,,
362725,-23.547403,-46.300659,2,2
583653,-23.547369,-46.358326,3,2


In [5]:
agg_df = (
    pd.concat([min_dist_df, count_df], axis="columns")
    # some polling stations are not close enough to any aid locations
    .fillna(0)
)

agg_df.reset_index().rename_axis(None, axis=1).set_index("poll_id")

,poll_lat,poll_lon,p116170_min_dist,p125829_min_dist,p116170_count,p125829_count
poll_id,,,,,,
362725,-23.547403,-46.300659,34.172744,41.607432,2,2
583653,-23.547369,-46.358326,28.298120,37.446734,3,2


In [6]:
agg_df.to_csv("data/demo_design.csv")